## Setup

This guide was written in Python 3.5.

### Libraries

We'll be working with the re library for regular expressions and nltk for natural language processing techniques, so make sure to install them! To install these libraries, enter the following commands into your terminal: 

``` 
pip3 install nltk
pip3 install spacy
pip3 install pandas
pip3 install scikit-learn
pip3 install sklearn
```

### Other

Sentence boundary detection requires the dependency parse, which requires data to be installed, so enter the following command in your terminal. 

```
python3 -m spacy.en.download all
```

## Background

### Polarity Flippers

Polarity flippers are words that change positive expressions into negative ones or vice versa. 

#### Negation 

Negations directly change an expression's sentiment by preceding the word before it. An example would be

```
The cat is not nice.
```

#### Constructive Discourse Connectives

Constructive Discourse Connectives are words which indirectly change an expression's meaning with words like "but". An example would be 

``` 
I usually like cats, but this cat is evil.
```

### Multiword Expressions

Multiword expressions are important because, depending on the context, can be considered positive or negative. For example, 

``` 
This song is shit.
```
is definitely considered negative. Whereas

``` 
This song is the shit.
```
is actually considered positive, simply because of the addition of 'the' before the word 'shit'.

### WordNet

WordNet is an English lexical database with emphasis on synonymy - sort of like a thesaurus. Specifically, nouns, verbs, adjectives and adjectives are grouped into synonym sets. 

#### Synsets

nltk has a built-in WordNet that we can use to find synonyms. We import it as such:

In [25]:
from nltk.corpus import wordnet as wn

If we feed a word to the synsets() method, the return value will be the class to which belongs. For example, if we call the method on motorcycle,  


In [28]:
print(wn.synsets('dog'))

[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01'), Synset('chase.v.01')]


Awesome stuff! But if we want to take it a step further, we can. We've previously learned what lemmas are - if you want to obtain the lemmas for a given synonym set, you can use the following method:


In [4]:
print(wn.synset('car.n.01').lemma_names())


['car', 'auto', 'automobile', 'machine', 'motorcar']


Even more, you can do things like get the definition of a word: 


In [5]:
print(wn.synset('car.n.01').definition())


a motor vehicle with four wheels; usually propelled by an internal combustion engine


#### Negation

With WordNet, we can easily detect negations. This is great because it's not only fast, but it requires no training data and has a fairly good predictive accuracy. On the other hand, it's not able to handle context well or work with multiple word phrases. 


### SentiWordNet

Based on WordNet synsets, SentiWordNet is a lexical resource for opinion mining, where each synset is assigned three sentiment scores: positivity, negativity, and objectivity.

In [29]:
from nltk.corpus import sentiwordnet as swn
cat = swn.senti_synset('dog.n.01')

In [30]:
cat.pos_score()

0.0

In [31]:
cat.neg_score()

0.0

In [32]:
cat.obj_score()

1.0

### Stop Words

Stop words are extremely common words that would be of little value in our analysis are often excluded from the vocabulary entirely. Some common examples are determiners like the, a, an, another, but your list of stop words (or <b>stop list</b>) depends on the context of the problem you're working on. 

### Testing

#### Cross Validation

Cross validation is a model evaluation method that works by not using the entire data set when training the model, i.e. some of the data is removed before training begins. Once training is completed, the removed data is used to test the performance of the learned model on this data. This is important because it prevents your model from over learning (or overfitting) your data. 

#### Precision

Precision is the percentage of retrieved instances that are relevant - it measures the exactness of a classifier. A higher precision means less false positives, while a lower precision means more false positives. 

#### Recall

Recall is the percentage of relevant instances that are retrieved. Higher recall means less false negatives, while lower recall means more false negatives. Improving recall can often decrease precision because it gets increasingly harder to be precise as the sample space increases.

#### F-measure 

The f1-score is a measure of a test's accuracy that considers both the precision and the recall. 

### Logistic Regression

Logistic regression is a generalized linear model commonly used for classifying binary data. Its output is a continuous range of values between 0 and 1, usually representing the probability, and its input is some form of discrete predictor. 

## Named Entity Extraction

Named entities are noun phrases that refer to specific types of individuals, such as organizations, people, dates, etc. Therefore, the purpose of a named entity recognition (NER) system is to identify all textual mentions of the named entities.

### spaCy

In the following exercise, we'll build our own named entity recognition system with the Python module `spaCy`, a Python module commonly used for Natural Language Processing in industry. 

In [9]:
import spacy
import pandas as pd

Using spaCy, we'll load the built-in English tokenizer, tagger, parser, NER and word vectors. We indicate this with the parameter `'en'`:

In [10]:
nlp = spacy.load('en')

In [11]:
review = "Columbia University was founded in 1754 as King's College by royal charter of King George II of England. It is the oldest institution of higher learning in the state of New York and the fifth oldest in the United States. Controversy preceded the founding of the College, with various groups competing to determine its location and religious affiliation. Advocates of New York City met with success on the first point, while the Anglicans prevailed on the latter. However, all constituencies agreed to commit themselves to principles of religious liberty in establishing the policies of the College. In July 1754, Samuel Johnson held the first classes in a new schoolhouse adjoining Trinity Church, located on what is now lower Broadway in Manhattan. There were eight students in the class. At King's College, the future leaders of colonial society could receive an education designed to 'enlarge the Mind, improve the Understanding, polish the whole Man, and qualify them to support the brightest Characters in all the elevated stations in life.'' One early manifestation of the institution's lofty goals was the establishment in 1767 of the first American medical school to grant the M.D. degree."

In [12]:
doc = nlp(review)

Going along the process of named entity extraction, we begin by segmenting the text, i.e. splitting it into a list of sentences. 

In [13]:
sentences = [sentence.orth_ for sentence in doc.sents] # list of sentences
print("There were {} sentences found.".format(len(sentences)))

There were 9 sentences found.


Now, we go a step further, and count the number of nounphrases by taking advantage of chunk properties.

In [14]:
nounphrases = [[np.orth_, np.root.head.orth_] for np in doc.noun_chunks]
print("There were {} noun phrases found.".format(len(nounphrases)))

There were 52 noun phrases found.


Lastly, we achieve our final goal: entity extraction. 

In [15]:
entities = list(doc.ents) # converts entities into a list
print("There were {} entities found".format(len(entities)))

There were 26 entities found


So now, we can turn this into a DataFrame for better visualization: 

In [16]:
orgs_and_people = [entity.orth_ for entity in entities if entity.label_ in ['ORG','PERSON']]
pd.DataFrame(orgs_and_people)

,0
0,Columbia University
1,King's College
2,George II
3,College
4,College
5,Samuel Johnson
6,Trinity Church
7,King's College


nsurprisingly, Columbia University is an entity, along with other names like King's College and Samuel Johnson.

In summary, named entity extraction typically follows the process of sentence segmentation, noun phrase chunking, and, finally, entity extraction. 

### nltk

Next, we'll work through a similar example as before, this time using the nltk module to extract the named entities through the use of chunk parsing. As always, we begin by importing our needed modules and example: 

In [12]:
import nltk
import re
content = "Starbucks has not been doing well lately"

Then, as always, we tokenize the sentence and follow up with parts-of-speech tagging. 

In [13]:
tokenized = nltk.word_tokenize(content)
tagged = nltk.pos_tag(tokenized)
print(tagged)

[('Starbucks', 'NNP'), ('has', 'VBZ'), ('not', 'RB'), ('been', 'VBN'), ('doing', 'VBG'), ('well', 'RB'), ('lately', 'RB')]


Great, now we've got something to work with! 

``` 
[('Starbucks', 'NNP'), ('has', 'VBZ'), ('not', 'RB'), ('been', 'VBN'), ('doing', 'VBG'), ('well', 'RB'), ('lately', 'RB')]
```

So we take this POS tagged sentence and feed it to the `nltk.ne_chunk()` method. This method returns a nested Tree object, so we display the content with namedEnt.draw(). 

In [14]:
namedEnt = nltk.ne_chunk(tagged)
namedEnt.draw()

Now, if you wanted to simply get the named entities from the namedEnt object we created, how do you think you would go about doing so?

## Information Extraction

Information Extraction is the process of acquiring meaning from text in a computational manner. 

### Data Forms

#### Structured Data

Structured Data is when there is a regular and predictable organization of entities and relationships.

#### Unstructured Data

Unstructured data, as the name suggests, assumes no organization. This is the case with most written textual data. 

### What is Information Extraction?

With that said, information extraction is the means by which you acquire structured data from a given unstructured dataset. There are a number of ways in which this can be done, but generally, information extraction consists of searching for specific types of entities and relationships between those entities. 

An example is being given the following text, 

```
Martin received a 98% on his math exam, whereas Jacob received a 84%. Eli, who also took the same test, received an 89%. Lastly, Ojas received a 72%.
```
This is clearly unstructured. It requires reading for any logical relationships to be extracted. Through the use of information extraction techniques, however, we could output structured data such as the following: 

```
Name     Grade
Martin   98
Jacob    84
Eli      89
Ojas     72
```

## Chunking

Chunking is used for entity recognition and segments and labels multitoken sequences. This typically involves segmenting multi-token sequences and labeling them with entity types, such as 'person', 'organization', or 'time'. 

### Noun Phrase Chunking

Noun Phrase Chunking, or NP-Chunking, is where we search for chunks corresponding to individual noun phrases.

We can use nltk, as is the case most of the time, to create a chunk parser. We begin with importing nltk and defining a sentence with its parts-of-speeches tagged (which we covered in the previous tutorial). 


In [11]:
import nltk 
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"), ("dog", "NN"), ("barked", "VBD"), ("at", "IN"), ("the", "DT"), ("cat", "NN")]

Next, we define the tag pattern of an NP chunk. A tag pattern is a sequence of part-of-speech tags delimited using angle brackets, e.g. `<DT>?<JJ>*<NN>`. This is how the parse tree for a given sentence is acquired.  


In [12]:
pattern = "NP: {<DT>?<JJ>*<NN>}" 

Finally we create the chunk parser with the nltk RegexpParser() class. 


In [13]:
NPChunker = nltk.RegexpParser(pattern) 

And lastly, we actually parse the example sentence and display its parse tree. 


In [14]:
result = NPChunker.parse(sentence) 
result.draw()

## Relation Extraction 

Once we have identified named entities in a text, we then want to analyze for the relations that exist between them. This can be performed using either rule-based systems, which typically look for specific patterns in the text that connect entities and the intervening words, or using machine learning systems that typically attempt to learn such patterns automatically from a training corpus.

### Rule-Based Systems

In the rule-based systems approach, we look for all triples of the form (X, a, Y), where X and Y are named entities and a is the string of words that indicates the relationship between X and Y. Using regular expressions, we can pull out those instances of a that express the relation that we are looking for. 

In the following code, we search for strings that contain the word "in". The special regular expression `(?!\b.+ing\b)` allows us to disregard strings such as `success in supervising the transition of`, where "in" is followed by a gerund. 

In [15]:
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
    for rel in nltk.sem.relextract.extract_rels('ORG', 'LOC', doc,corpus='ieer', pattern = IN):
         print (nltk.sem.relextract.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


And so we get: 

```
[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']
```

Note that the X and Y named entitities types all match with one another! Object type matching is an important and required part of this process. 

## Sentiment Analysis

As we saw in the previous tutorial, sentiment analysis refers to the use of text analysis and statistical learning to identify and extract subjective information in textual data. For our last exercise in this tutorial, we'll introduce and use linear models in the context of a sentiment analysis problem.

### Loading the Data

First, we begin by loading the data. Since we'll be using data available online, we'll use the urllib module to avoid having to manually download any data.

In [16]:
import urllib.request

So then we'll define the test and training data URLs to variables, as well as filenames for each of those datasets.

In [17]:
test_url = "https://dl.dropboxusercontent.com/u/8082731/datasets/UMICH-SI650/testdata.txt"
train_url = "https://dl.dropboxusercontent.com/u/8082731/datasets/UMICH-SI650/training.txt"

test_file = 'test_data.csv'
train_file = 'train_data.csv'

Using the links and filenames from above, we'll officially download the data using the urlib.request.urlretrieve method. 

In [18]:
test_data_f = urllib.request.urlretrieve(test_url, test_file)
train_data_f = urllib.request.urlretrieve(train_url, train_file)

Now that we've downloaded our datasets, we can load them into pandas dataframes. First for the test data:

In [19]:
import pandas as pd

test_data_df = pd.read_csv(test_file, header=None, delimiter="\t", quoting=3)
test_data_df.columns = ["Text"]

Next for training data: 

In [20]:
train_data_df = pd.read_csv(train_file, header=None, delimiter="\t", quoting=3)
train_data_df.columns = ["Sentiment","Text"]

Just to see how the dataframe looks, let's call the .head() method on both dataframes. 

In [21]:
test_data_df.head()

,Text
0,""" I don't care what anyone says, I like Hillar..."
1,have an awesome time at purdue!..
2,"Yep, I'm still in London, which is pretty awes..."
3,"Have to say, I hate Paris Hilton's behavior bu..."
4,i will love the lakers.


And we get: 

```
                                                Text
0  " I don't care what anyone says, I like Hillar...
1                  have an awesome time at purdue!..
2  Yep, I'm still in London, which is pretty awes...
3  Have to say, I hate Paris Hilton's behavior bu...
4                            i will love the lakers.
```


In [22]:
train_data_df.head()

,Sentiment,Text
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.
3,1,i liked the Da Vinci Code a lot.
4,1,I liked the Da Vinci Code but it ultimatly did...


And we get:

``` 
   Sentiment                                               Text
0          1            The Da Vinci Code book is just awesome.
1          1  this was the first clive cussler i've ever rea...
2          1                   i liked the Da Vinci Code a lot.
3          1                   i liked the Da Vinci Code a lot.
4          1  I liked the Da Vinci Code but it ultimatly did...
```

### Preparing the Data

To implement our bag-of-words linear classifier, we need our data in a format that allows us to feed it in to the classifer. Using sklearn.feature_extraction.text.CountVectorizer in the Python scikit learn module, we can convert the text documents to a matrix of token counts. So first, we import all the needed modules: 

In [23]:
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer

We need to remove punctuations, lowercase, remove stop words, and stem words. All these steps can be directly performed by CountVectorizer if we pass the right parameter values. We can do this as follows. 

We first create a stemmer, using the Porter Stemmer implementation.

In [24]:
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = [stemmer.stem(item) for item in tokens]
    return(stemmed)

Here, we have our tokenizer, which removes non-letters and stems:

In [25]:
def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return(stems)

Here we init the vectoriser with the CountVectorizer class, making sure to pass our tokenizer and stemmers as parameters, remove stop words, and lowercase all characters.

In [26]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = 'english',
    max_features = 85
)

Next, we use the `fit_transform()` method to transform our corpus data into feature vectors. Since the input needed is a list of strings, we concatenate all of our training and test data. 

In [27]:
features = vectorizer.fit_transform(
    train_data_df.Text.tolist() + test_data_df.Text.tolist())

Here, we're simply converting the features to an array for easier use.

In [28]:
features_nd = features.toarray()

### Linear Classifier

Finally, we begin building our classifier. Earlier we learned what a bag-of-words model. Here, we'll be using a similar model, but with some modifications. To refresh your mind, this kind of model simplifies text to a multi-set of terms frequencies. 

So first we'll split our training data to get an evaluation set. As we mentioned before, we'll use cross validation to split the data. sklearn has a built-in method that will do this for us. All we need to do is provide the data and assign a training percentage (in this case, 75%).

In [29]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd[0:len(train_data_df)], 
        train_data_df.Sentiment,
        train_size=0.85, 
        random_state=1234)

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Now we're ready to train our classifier. We'll be using Logistic Regression to model this data. Once again, sklearn has a built-in model for you to use, so we begin by importing the needed modules and calling the class.  

In [30]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

And as always, we need actually do the training, so we call the `.fit()` method on our data. 

In [31]:
log_model = log_model.fit(X=X_train, y=y_train)

Now we use the classifier to label the evaluation set we created earlier:

In [32]:
y_pred = log_model.predict(X_test)

You can see that this array of labels looks like: 

```
array([0, 1, 0, ..., 0, 1, 0])
```

### Accuracy

In sklearn, there is a function called sklearn.metrics.classification_report which calculates several types of predictive scores on a classification model. So here we check out how exactly our model is performing:

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.98      0.99      0.98       467
          1       0.99      0.98      0.99       596

avg / total       0.98      0.98      0.98      1063



And we get: 

```
                 precision    recall  f1-score   support

              0       0.98      0.99      0.98       467
              1       0.99      0.98      0.99       596

    avg / total       0.98      0.98      0.98      1063
```

where precision, recall, and f1-score are the accuracy values discussed in the section 1.6. Support is the number of occurrences of each class in y_true and x_true.

### Retraining 

Finally, we can re-train our model with all the training data and use it for sentiment classification with the original unlabeled test set. 

So we repeat the process from earlier, this time with different data:

In [34]:
log_model = LogisticRegression()
log_model = log_model.fit(X=features_nd[0:len(train_data_df)], y=train_data_df.Sentiment)
test_pred = log_model.predict(features_nd[len(train_data_df):])

So again, we can see what the predictions look: 

```
array([1, 1, 1, ..., 1, 1, 0])
```

And lastly, let's actually look at our predictions! Using the random module to select a random sliver of the data we predicted on, we'll print the results. 

In [35]:
import random
spl = random.sample(range(len(test_pred)), 10)
for text, sentiment in zip(test_data_df.Text[spl], test_pred[spl]):
    print (sentiment, text)

1 I like my HP Pavilion Laptop.
1 i miss AAA...
0 London yesterday kinda sucked tbh, well, the Houses of Parliament did anyway.
0 TAKE THAT STUPID UCLA!!!!!!!..
1 If God exists, why are morons like Paris Hilton considered beautiful?.(
1 I absolutely love my MacBook Pro.
1 I loved Boston and MIT so much and still do.
0 And as stupid as San Francisco's road system is, we weren't able to turn back because of how all the roads are one-way streets.
0 usually, i can't at my house but i'm at the library but i moved into my aunts house in san carlos to go to school here because schools in san francisco sucks BIG TIME!
0 Way to go stupid Lakers..


Recall that 0 indicates a negative sentence and 1 indicates a positive: